## Using a benchmark problem
This tutorial shows how to use and query a benchmark problem.

In [ ]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

**There a 6 benchmarks in total:** 3 scenarios, each with sequence or path variant. The sequence variants have dense Jacobians and optimize over a discrete sequence of few (maybe one) configurations; the path variants optimize over a whole continuous path and have sparse Jacobians.

*All problems neglect collisions.* Esp. the last one has lots of collisions. But there are other difficulties in the last one.

The following loads a benchmark problem, from which we get the ry.MathematicalProgram interface

In [ ]:
benchmark = ry.OptBench_Skeleton_Pick(ry.arg.path)  #1. scenario: simple pick, just one configuration or path
#benchmark = ry.OptBench_Skeleton_Handover(ry.arg.path) #2. scenario: pick-handover-touch, 3 configurations or path
#benchmark = ry.OptBench_Skeleton_StackAndBalance(ry.arg.sequence) #3. scenario: pick boxes and stack
nlp = benchmark.get()

In [ ]:
# If you want, get a full API:
#help(nlp)

The first method to use is `getInitializationSample`, which returns an initial x

In [ ]:
x = nlp.getInitializationSample()

The most important method is `evaluate(x)`, which returns a tuple `(phi,J)`

In [ ]:
(phi,J) = nlp.evaluate(x)
print('phi=', phi, '\nJ=', J)

Note the dimensionalities:

In [ ]:
print('dim(x)=', nlp.getDimension())
print('dim(x)=', x.shape)
print('dim(phi)=', phi.shape)
print('dim(J)=', J.shape)

The features `phi` define all cost terms and inequality/equality constraint values. The feature types tell you, which feature entries refer to costs/constraints. For each feature, an integer is returned -- but actually it is an enum. So you can test equality with ry.OT.f, ry.OT.sos, ry.OT.ineq, or ry.OT.eq

In [ ]:
types = nlp.getFeatureTypes()
print(types)
print(types[0] == ry.OT.f)
print(types[0] == ry.OT.sos)
print(types[0] == ry.OT.ineq)
print(types[0] == ry.OT.eq)

In the benchmarks, most features are features are sum-of-square features (e.g. from control costs). Some are equality constraints.

Finally, the benchmark problems implement a report method, which should help you to get more insight in what $x$ means and what is computed. For high verbosity it also displays something - in our case the robot in pose $x$.

**Hit ENTER in the displayed window** to continue.

In [ ]:
# the report method generally returns some info string - internal information given by the benchmark implementation
print(nlp.report(4))

## demo for testing
Calling the rai optimizer as below is just for testing - you're supposed to instead implement your own solvers!

In [ ]:
#setup a solver with that problem
solver = ry.NLP_Solver()
solver.setProblem(nlp)
solver.setSolver(ry.NLP_SolverID.augmentedLag)

In [ ]:
#solve it
solver.solve(True)

In [ ]:
# the trace of queries
solver.getTrace_x()

In [ ]:
#the trace of objectives for each query: the first is total cost, 2nd is ineq, 3rd is eq
solver.getTrace_costs()

In [ ]:
print(nlp.report(6))

In [ ]:
nlp = 0
benchmark = 0
solver = 0